1.1 GDP

In [7]:
import requests
import pandas as pd
import json
from prettytable import PrettyTable
import streamlit as st
# from data import utils

In [8]:
@st.cache_data

def fetch_data(series_id):
    
    """Fetch observations for a FRED series and save to a DataFrame."""
    url = f"https://api.stlouisfed.org/fred/series/observations"
    
    api_key = "9648695895d3facac4fba9c7cb834427"
    
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json"
    }
    response = requests.get(url, params=params)
    

    data = response.json()
    observations = data.get('observations', [])
    # Create DataFrame
    df = pd.DataFrame(observations)
    # Display the DataFrame
    
    return df

2024-06-22 13:54:03.221 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [17]:

def get_ir_10yr():

    # Interest Rates: Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for United States
    series_id = "IR3TIB01USM156N"

    df = fetch_data(series_id)
    df = df[['date','value']]
    df.rename(columns = {'date':'Month', 'value': 'Interest_Rate_10yr_Bond'}, inplace= True)
    
    return df

In [18]:
df = get_ir_10yr()
df

,Month,Interest_Rate_10yr_Bond
0,1964-06-01,3.86
1,1964-07-01,3.87
2,1964-08-01,3.85
3,1964-09-01,3.87
4,1964-10-01,3.94
...,...,...
715,2024-01-01,5.26
716,2024-02-01,5.22
717,2024-03-01,5.29
718,2024-04-01,5.33


In [ ]:


# st.set_page_config(initial_sidebar_state="collapsed")
# Set the page config at the start of the main script
st.set_page_config(
    page_title="Dashboard",
    page_icon="🏂",
    layout="wide",
    initial_sidebar_state="collapsed"
)


pages = ["Indicators", "Stock Index", "Knowledge", "Resources", "GitHub"]

parent_dir = os.path.dirname(os.path.abspath(__file__))

# logo_path = os.path.join(parent_dir, "cubes.svg")

urls = {"GitHub": "https://github.com/double-tea-l/ecom-indicator-dash/"}

styles = {
    "nav": {
        "background-color": "royalblue",
        "justify-content": "left",
    },
    "img": {
        "padding-right": "14px",
    },
    "span": {
        "color": "white",
        "padding": "14px",
    },
    "active": {
        "background-color": "white",
        # "color": "var(--text-color)",
        "color": "black",
        "font-weight": "normal",
        "padding": "14px",
    }
}

options = {
    "show_menu": False,
    "show_sidebar": False,
}

page = st_navbar(
    pages,
    # logo_path=logo_path,
    urls=urls,
    styles=styles,
    options=options,
)

functions = {
    "Indicators": pg.show_indicators,
    # "Stock Index": pg.show_stocks
}
go_to = functions.get(page)
if go_to:
    go_to()

In [1]:
def get_gdp(freq, series_id): 

    # Define the URL and parameters
    url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'

    param = [('dataset', 'IFS'),
            ('freq', freq),
            ('country', 'US'),
            ('series', series_id), 
            ('start', '?startPeriod=1920')]


    series = '.'.join([i[1] for i in param[1:4]])

    key = f'CompactData/{param[0][1]}/{series}{param[-1][1]}'

    # Combine API url with key specific to data request
    r = requests.get(f'{url}{key}').json()

    # Extract the data portion of results
    data = r['CompactData']['DataSet']['Series']

    # Check if data is a list or a single dictionary
    if isinstance(data, dict):
        data = [data]

    # Create pandas dataframe, column = country, row = obs
    df = pd.DataFrame({s['@REF_AREA']:
                    {i['@TIME_PERIOD']: float(i['@OBS_VALUE']) 
                        for i in s['Obs']} for s in data})

    # Convert index to datetime
    df.index = pd.to_datetime(df.index)
    
    return df

1.2 CPI and PPI

https://data.bls.gov/cgi-bin/surveymost?bls


In [2]:
# series_id = ['WPSFD4'] #'CUUR0000SA0', 'WPSFD4','EIUIR','EIUIQ','LNS14000000' # PPI, CPI, Import, Export, Unemployment
start_year = '1920'
end_year = '2024'

def get_bls_data(series_id, start_year, end_year): 
    
    headers = {'Content-type': 'application/json'}
    
    series_data = json.dumps({"seriesid": [series_id] ,"startyear" : start_year, "endyear" : end_year})  

    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data = series_data, headers = headers)
    json_data = json.loads(p.text)
    for series in json_data['Results']['series']:
        x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
        seriesId = series['seriesID']
        for item in series['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','
            if 'M01' <= period <= 'M12':
                x.add_row([seriesId,year,period,value,footnotes[0:-1]])
        output = open(seriesId + '.txt','w')
        output.write (x.get_string())
        output.close()
        
    # Extract the relevant data
    series_data = json_data['Results']['series'][0]['data']
    # Convert to DataFrame

    df = pd.DataFrame(series_data)

    return df
    

In [3]:
# CPI
df_cpi = get_bls_data('WPSFD4', start_year, end_year)

# PPI
df_ppi = get_bls_data('CUUR0000SA0', start_year, end_year)

# unemployment rate
df_unemployment_rate = get_bls_data('LNS14000000', start_year, end_year) 

NameError: name 'json' is not defined

In [ ]:
df_ppi

,year,period,periodName,value,footnotes
0,1929,M12,December,17.2,[{}]
1,1929,M11,November,17.3,[{}]
2,1929,M10,October,17.3,[{}]
3,1929,M09,September,17.3,[{}]
4,1929,M08,August,17.3,[{}]
...,...,...,...,...,...
115,1920,M05,May,20.6,[{}]
116,1920,M04,April,20.3,[{}]
117,1920,M03,March,19.7,[{}]
118,1920,M02,February,19.5,[{}]


In [ ]:
series_id = 'NGDP_XDC' # Nominal GDP in national currency 
freq = 'A'

df_gdp = get_gdp(freq, series_id)

In [ ]:

import pandas as pd
import math
from prettytable import PrettyTable
from ..data import indicators_data as indicators



# def df_gdp():
    
#     i = indicators.get_major_indicators()
    
#     #inputs
#     us_gdp_pct_change = i.gdp_pct_change
#     us_gdp = i.gdp
#     us_real_gdp = i.real_gdp
     
#     # format
#     df_gdp = pd.merge(us_gdp_pct_change, us_gdp, on = 'date', how = 'outer')
#     df_gdp = pd.merge(df_gdp, us_real_gdp, on = 'date', how = 'outer')
#     df_gdp = df_gdp.sort_values(by='date', ascending= True).reset_index(drop = True)
#     df_gdp['date'] = pd.to_datetime(df_gdp['date'])
#     # Function to convert date to quarter
#     def date_to_quarter(date):
#         return f"{date.year} Q{((date.month - 1) // 3) + 1}"
#     # Apply the function to the dataframe
#     df_gdp['Quarter'] = df_gdp['date'].apply(date_to_quarter)
#     df_gdp = df_gdp[df_gdp['date']>='1948-01-01']
#     # Convert 'GDP' and 'Real_GDP' columns to numeric, coercing errors
#     df_gdp['GDP'] = pd.to_numeric(df_gdp['GDP'], errors='coerce')
#     df_gdp['Real_GDP'] = pd.to_numeric(df_gdp['Real_GDP'], errors='coerce')

#     # Round up GDP values to integers
#     df_gdp['GDP'] = df_gdp['GDP'].apply(lambda x: math.ceil(x) if pd.notnull(x) else x)
#     df_gdp['Real_GDP'] = df_gdp['Real_GDP'].apply(lambda x: math.ceil(x) if pd.notnull(x) else x)
    
#     return df_gdp


# def df_cpi_ppi():
    
i = indicators.get_major_indicators()
i.get_ir_10yr()

ImportError: attempted relative import with no known parent package

In [ ]:

import pandas as pd
import math
from prettytable import PrettyTable
from data import indicators_data as indicators

i = indicators.get_major_indicators()
cpi = i.cpi
ppi = i.ppi

cpi

ModuleNotFoundError: No module named 'data'